In [2]:
import nrrd
import numpy as np
import pandas as pd
import json
import scipy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchio as tio
import plotly.graph_objects as go

# Load the data
raster_data, raster_header = nrrd.read('cut/7_12-03-2018_cut.nrrd')
seg_data, _ = nrrd.read('segmented/7_12-03-2018_cut_seg.nrrd')

tbv = float(raster_header["tbv"])
spacings = raster_header["spacings"].astype(np.float32)

print(raster_data.shape)
print(raster_header)

/home/daru/code/tbv/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(251, 177, 217)
OrderedDict([('type', 'uint8'), ('dimension', 3), ('sizes', array([251, 177, 217])), ('kinds', ['space', 'space', 'space']), ('encoding', 'raw'), ('spacings', array([0.43508277, 0.43508277, 0.43508277])), ('labels', ['x', 'y', 'z']), ('units', ['cm', 'cm', 'cm']), ('tbv', '120.88'), ('age_days', '2')])


In [3]:
def prep_raster(raster, voxel_spacings, new_sizes=[50, 50, 50]):
    old_sizes = np.array(raster.shape)
    old_spacings = voxel_spacings
    new_sizes = np.array(new_sizes)

    if np.array_equal(old_sizes, new_sizes):
        return raster, voxel_spacings
    
    resize_factors = new_sizes / old_sizes

    raster = scipy.ndimage.zoom(raster, resize_factors)
    new_spacings = old_spacings / resize_factors

    return raster, new_spacings

norm_transform = tio.Compose([
    tio.transforms.RescaleIntensity(), 
    tio.transforms.ZNormalization()
])


In [4]:
l = 96

raster_data = raster_data[0:96, 0:96, 0:96]

vol, vol_spacings = prep_raster(raster_data, spacings, new_sizes=[l, l, l])
vol = norm_transform(torch.from_numpy(vol).unsqueeze(0).float()).squeeze(0).numpy()

vol = (vol - vol.min()) / (vol.max() - vol.min())

print(vol.shape)
print("Max:", np.max(vol))
print("Min:", np.min(vol))
print("Mean:", np.mean(vol))

(96, 96, 96)
(96, 96, 96)
Max: 1.0
Min: 0.0
Mean: 0.39395383


In [5]:
tbv_voxels_prev = tbv / np.prod(spacings)
tbv_voxels = tbv / np.prod(vol_spacings)

print("TBV voxels (prev):", tbv_voxels_prev)
print("TBV voxels (new):", tbv_voxels)
print("TBV:", tbv)

TBV voxels (prev): 1467.7051686307248
TBV voxels (new): 1467.7051686307248
TBV: 120.88


In [13]:
vol = scipy.ndimage.zoom(vol, (0.3, 0.3, 0.3))
X, Y, Z = np.mgrid[:vol.shape[0], :vol.shape[1], :vol.shape[2]]

fig = go.Figure(data=go.Volume(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
    value=vol.flatten(),
    isomin=0.2,
    isomax=1.0,
    opacity=0.2,
    surface_count=3,
    ))
fig.update_layout(scene_xaxis_showticklabels=False,
                scene_yaxis_showticklabels=False,
                scene_zaxis_showticklabels=False)
fig.show()

In [14]:
segmentation_down, _ = prep_raster(seg_data, spacings, new_sizes=[l, l, l])
segmentation_down = segmentation_down.astype(np.float32)
vol = (segmentation_down - segmentation_down.min()) / (segmentation_down.max() - segmentation_down.min())

print(vol.shape)
print("Max:", np.max(vol))
print("Min:", np.min(vol))
print("Mean:", np.mean(vol))

(96, 96, 96)
Max: 1.0
Min: 0.0
Mean: 0.14579937


In [15]:
vol = scipy.ndimage.zoom(vol, (0.3, 0.3, 0.3))
X, Y, Z = np.mgrid[:vol.shape[0], :vol.shape[1], :vol.shape[2]]

fig = go.Figure(data=go.Volume(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
    value=vol.flatten(),
    isomin=0.2,
    isomax=1.0,
    opacity=0.2,
    surface_count=3,
    ))
fig.update_layout(scene_xaxis_showticklabels=False,
                  scene_yaxis_showticklabels=False,
                  scene_zaxis_showticklabels=False)
fig.show()

In [16]:
voxels = vol[vol > 0.9].sum()
print("Voxels:", voxels)
print("TBV:", voxels * np.prod(vol_spacings)/1000)
print("Real TBV:", tbv)

Voxels: 3161.9292
TBV: 2.8376628764940883
Real TBV: 120.88
